In [1]:
import torch
import torchdata
import torchtext
from torch import nn
import time

#1. puffer - it's outdated
#2. spend some money - 300 baht get collab pro
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#reproducibility
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
torch.__version__

'1.13.1+cu117'

In [3]:
torchdata.__version__

'0.5.1'

In [4]:
torchtext.__version__

'0.14.1'

## 1. Load the dataset

Make our life easy by using some ready-to-be-used dataset by torchtext

- In your assignment, i will ask you to use penn treebank

In [5]:
#if you are using puffer
import sys
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

## 2.EDA - exploratory data analysis

- check common words
- look at some random sample.... how it looks, so that we can design power, neural network
- visualize statistic

In [6]:
# Load data
import pytreebank
import sys
import os
# dataset = pytreebank.load_sst()
# train = dataset['train']
# valid = dataset['dev']
# test = dataset['test']

In [7]:
train = pytreebank.import_tree_corpus("./sentiment/train.txt")
valid = pytreebank.import_tree_corpus("./sentiment/dev.txt")
test = pytreebank.import_tree_corpus("./sentiment/test.txt")

In [8]:
set([label for t in train for label, sentence in t.to_labeled_lines()])

{0, 1, 2, 3, 4}

In [9]:
train_size = len(list(iter(train)))
valid_size = len(list(iter(valid)))
test_size = len(list(iter(test)))

train_size,valid_size,test_size

(8544, 1101, 2210)

## 3. Preprocessing

- tokenizing, numericalization

In [10]:
## 3.1 Tokenizing

from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy',language='en_core_web_sm')
#check whether the tokenizer works....
# tokens = tokenizer('Chaky likes deep learning very much and wants his student to be nubmer 1 in Asia')
# tokens

In [11]:
## 3.2 Numericalization

from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter): #data_iter, e.g., train
    for data in data_iter:
        for _, text in data.to_labeled_lines():
            yield tokenizer(text)
    
vocab = build_vocab_from_iterator(yield_tokens(train), specials=['<unk>','<pad>','<bos>','<eos>'], special_first = True)

In [12]:
vocab.set_default_index(vocab["<unk>"]) #if you don't the id of this word, set it unk

In [13]:
vocab(['Chaky','likes','deep','learning','very','much','and','wants','his','student','to','be','nubmer','1','in','Asia'])

[0, 6138, 602, 5318, 85, 64, 8, 919, 36, 2733, 9, 28, 0, 3233, 14, 8506]

In [14]:
id2word = vocab.get_itos()

In [15]:
id2word[0]

'<unk>'

In [16]:
vocab(['<pad>','<bos>','<eos>'])

[1, 2, 3]

In [17]:
len(vocab) #23k unique words.......

17136

## 4. FastText embeddding

we gonna insert this embedding to the NN on the fly

In [18]:
from torchtext.vocab import FastText
fast_vectors = FastText(language='simple')

In [19]:
fast_embedding = fast_vectors.get_vecs_by_tokens(vocab.get_itos()).to(device)

In [20]:
fast_embedding.shape

torch.Size([17136, 300])

In [21]:
#please lookup the fasttext embedding of id 100
fast_embedding[100].shape

torch.Size([300])

In [22]:
fast_embedding[100][:10]

tensor([-0.0595, -0.0829,  0.0264, -0.2511, -0.0340,  0.2841,  0.1026, -0.0634,
         0.0880,  0.1869], device='cuda:0')

## 5. Preparing dataloader

Optional - you can either make your owen bacth loader....
you can use pytorch dataloader

In [23]:
# pad_sequence([1,2,3],[1,2,0])

In [24]:
label_pipeline = lambda x: int(x) - 1 #1,2,3,,4 --> 0,1,2,3
text_pipeline = lambda x: vocab(tokenizer(x))

In [25]:
'''
why padding ????
in the same batch, e.g., batch size = 2

"Chaky eat sushi" ==> "Chaky", "eat", "sushi" ==> 0,22,11
"Chaky sleep" ==> "Chaky", "sleep" ==> 0,99, <pad>
'''

'\nwhy padding ????\nin the same batch, e.g., batch size = 2\n\n"Chaky eat sushi" ==> "Chaky", "eat", "sushi" ==> 0,22,11\n"Chaky sleep" ==> "Chaky", "sleep" ==> 0,99, <pad>\n'

In [26]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence #mkaing each batch batch same length

pad_ix = vocab['<pad>']

#this function gonna be called by DataLoader
def collate_batch(batch):
    label_list, text_list, length_list = [],[],[]
    for b in batch:
        for (_label,_text) in b.to_labeled_lines():
            label_list.append(label_pipeline(_label)) # (3, 1, 0, 2, 1)
            processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64) #(0, 44, 25, 1)
            text_list.append(processed_text)
            length_list.append(processed_text.size(0)) #for padding
    return torch.tensor(label_list,dtype=torch.int64), \
        pad_sequence(text_list, padding_value=pad_ix,batch_first=True), \
        torch.tensor(length_list,dtype=torch.int64)

In [27]:
batcgh_size = 8

train_loader = DataLoader(train,batch_size=batcgh_size,
                            shuffle=True,collate_fn=collate_batch)
val_loader = DataLoader(valid,batch_size=batcgh_size,
                            shuffle=True,collate_fn=collate_batch)
test_loader = DataLoader(test,batch_size=batcgh_size,
                            shuffle=True,collate_fn=collate_batch)

In [28]:
# for label,text,length in train_loader:
#     break
#label : [batch_size,]
#text  : [batch_size, longest length of this batch] ==> [batch size, seq len] ==> [b,l]
#length: [batch_size,]
# label, text, length #why we need length --> we can later ignore padding......
# label.shape, text.shape, length.shape

## 6. Designing the model

In [29]:
class LSTM(nn.Module):
    def __init__(self,input_dim, emb_dim, hid_dim, output_dim, num_layers, bidirectional, dropout):
        super().__init__()
        #input dim = how many vocab you have
        #emb dim = 300 --> we use fasttext
        #padding_idx tells this lookup to ignore and just randomize....
        #<unk>, <bos>, <eos>
        self.embedding_layer = nn.Embedding(input_dim, emb_dim, padding_idx=pad_ix)
        self.lstm = nn.LSTM(emb_dim, 
                            hid_dim, 
                            num_layers = num_layers, 
                            bidirectional = bidirectional, 
                            dropout = dropout, #dropout is applied between layers
                            batch_first = True) 
        self.fc = nn.Linear(hid_dim*2, output_dim)
    def forward(self, x, lengths):

        #x : [batch size, seq len]
        embedded_x = self.embedding_layer(x)
        #x : [batch size, seq len, emb dim]

        #pack this embedded_x in such a way that RNN knows to ignore padding....
        #without batch first = True; thing will become [seq len, batch size, emb size]
        pack_embedded = nn.utils.rnn.pack_padded_sequence(embedded_x, 
                                                            lengths.to('cpu'),
                                                            enforce_sorted=False,
                                                            batch_first=True)
        #packed_output is basically all hidden states ==> POS tagging, NER

        #h is the last hidden state => sentiment analysis
        #c is the last cell state
        packed_output, (h,c) = self.lstm(pack_embedded)

        #h : [num_layers * num_directions, batch_size, hidden dim] 

        #it happen that because packed_ouputs is all hidden states... some hidden state near the end is
        #hidden state for padding, pytorch guys help you
        #by using this pack_padded_sequence, them all the hiddden states will only be not padding
        output, output_length = nn.utils.rnn.pad_packed_sequence(packed_output,batch_first=True)

        #last hidden state - concat forward and backward states
        last_hidden_state = torch.cat((h[-1,:,:], h[-2,:,:]), dim = 1)
        #last_hidden_state: [batch_size, hidden_dim * 2]

        #for sentiment analysis... what whould i sent to my linear layer.....
        return self.fc(last_hidden_state) #[batch_size, output_dim] ==> [batch_size,4]

## 7. Training

In [30]:
#explicitly initialize weights for better learning
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.orthogonal_(param)

In [31]:
input_dim  = len(vocab)
hid_dim    = 256
emb_dim    = 300         #**<----change to 300
output_dim = 5 #four classes

#for biLSTM
num_layers = 2
bidirectional = True
dropout = 0.5

model = LSTM(input_dim, emb_dim, hid_dim, output_dim, num_layers, bidirectional, dropout).to(device)
model.apply(initialize_weights)
# model.embedding.weight.data = fast_embedding #**<------applied the fast text embedding as the initial weights

LSTM(
  (embedding_layer): Embedding(17136, 300, padding_idx=1)
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)

In [32]:
#we can print the complexity by the number of parameters
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model)

5140800
307200
262144
  1024
  1024
307200
262144
  1024
  1024
524288
262144
  1024
  1024
524288
262144
  1024
  1024
  2560
     5
______
7863109


In [33]:
import torch.optim as optim

lr=1e-3

#training hyperparameters
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss() #combine softmax with cross entropy

In [34]:
def accuracy(preds, y):
    
    predicted = torch.max(preds.data, 1)[1]
    batch_corr = (predicted == y).sum()
    acc = batch_corr / len(y)
    
    return acc

In [35]:
def train(model, loader, optimizer, criterion, loader_length):
    epoch_loss = 0
    epoch_acc = 0
    model.train() #useful for batchnorm and dropout
    
    for i, (label, text, text_length) in enumerate(loader): 
        label = label.to(device) #(batch_size, )
        text = text.to(device) #(batch_size, seq len)
                
        #predict
        predictions = model(text, text_length).squeeze(1) #output by the fc is (batch_size, 1), thus need to remove this 1
        
        #calculate loss
        loss = criterion(predictions, label)
        acc  = accuracy(predictions, label)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
                        
    return epoch_loss / loader_length, epoch_acc / loader_length

In [36]:
def evaluate(model, loader, criterion, loader_length):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for i, (label, text, text_length) in enumerate(loader): 
            label = label.to(device) #(batch_size, )
            text  = text.to(device)  #(seq len, batch_size)

            predictions = model(text, text_length).squeeze(1) 
            
            loss = criterion(predictions, label)
            acc  = accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / loader_length, epoch_acc / loader_length

In [37]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
best_valid_loss = float('inf')
num_epochs      = 5

save_path = f'models/{model.__class__.__name__}_TreeBank.pt'

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(num_epochs):
    
    start_time = time.time()

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, train_size)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion, valid_size)
    
    #for plotting
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [15,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)`

Nice!  Very high accuracy!

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(train_losses, label = 'train loss')
ax.plot(valid_losses, label = 'valid loss')
plt.legend()
ax.set_xlabel('updates')
ax.set_ylabel('loss')

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(train_accs, label = 'train acc')
ax.plot(valid_accs, label = 'valid acc')
plt.legend()
ax.set_xlabel('updates')
ax.set_ylabel('acc')

In [ ]:
model.load_state_dict(torch.load(save_path))

test_loss, test_acc = evaluate(model, test_loader, criterion, test_size)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## 8. Test on some random news

In [ ]:
test_str = "Google is now falling nonstop.  The price is really bad now."
text = torch.tensor(text_pipeline(test_str)).to(device)
text

In [ ]:
text_list = [x.item() for x in text]
text_list

In [ ]:
[id2word[num] for num in text_list]

In [ ]:
text = text.reshape(1, -1)  #because batch_size is 1

In [ ]:
text_length = torch.tensor([text.size(1)]).to(dtype=torch.int64)

In [ ]:
text.shape

In [ ]:
def predict(text, text_length):
    with torch.no_grad():
        output = model(text, text_length).squeeze(1)
        predicted = torch.max(output.data, 1)[1]
        return predicted

In [ ]:
predict(text, text_length)